In [32]:
# Library installer
!pip install googletrans
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# Loading Libraries
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from googletrans import Translator
from langdetect import detect_langs
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
from geopy.geocoders import Nominatim


In [59]:
# Constants
df_gemi = pd.read_csv('/content/elevatedetailed_v2.csv')
df_elevate = pd.read_excel('/content/gemi.xlsx')

# Data Preprocessing

In [60]:
# Merging the data files
df_elevate_gemi = pd.merge(df_gemi, df_elevate, on='id', how='left')

In [61]:
# Replace blank space with _ 
# put your dataframes into a list

for col in df_elevate_gemi.columns:
  # check if the column name contains a space
  if ' ' in col:

    # rename the column by replacing spaces with underscores
    df_elevate_gemi.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [62]:
df_elevate_gemi.head(2)

,Unnamed:_0.1,Unnamed:_0_x,id,Corporate_Name_x,Region,INDUSTRIES,TECHNOLOGY,BUSINESS_MODEL,REVENUE_MODEL,FUNDING_SOURCES,...,Unnamed:_0_y,Corporate_Name_y,Commercial_Titles,Date_Started,Legal_Type,Address,Website,kind_of_shares,no_of_shares,value_of_share
0,0,0,1,[i2.d] technologies,"Halandri, Greece",['Enterprise Software'],['Cloud Computing'],"['B2B', 'B2B2C']","['Commission Fee', 'SaaS', 'Subscription']",['VC'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,10,A.D.W.K. ADWORK,"ZOGRAFOU , Greece",['Advertising & Marketing (AdTech)'],"['AI', 'AR/VR', 'Data Analytics - Big Data', '...","['B2B', 'B2B2C']","['Price per Item', 'SaaS']",['Self'],...,0.0,A.D.W.K. ADWORK ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚΗ ΕΤΑΙΡΕΙΑ,ADWORK PC (Latin characters interpretation)ADW...,19/11/2021,PC,"XIROGIANNI 37, ATHINA, 15771",NaN,Capital,"1,600",€10.00





# Handling List




In [63]:
def clean_and_encode(df, col_name):
    unique_values = df[col_name].unique()
    print(unique_values)
    null_values = df[col_name].isnull()
    print(null_values.sum())
    df[col_name + '_updated'] = df[col_name].replace('[]', '''['Other']''')
    unique_values_update = df[col_name + '_updated'].unique()
    print(unique_values_update)
    # convert the data to a list of lists
    df_list = df[col_name + '_updated'].apply(lambda x: eval(x)).tolist()
    y = [ [col_name +'_'+ elem for elem in inner_list] for inner_list in df_list]
    print("********************** Printing DF LIST*************", y)
    # use MultiLabelBinarizer to perform one-hot encoding
    mlb = MultiLabelBinarizer()
    encoded_data = mlb.fit_transform(df_list)
    # create a new dataframe with the encoded data
    encoded_df = pd.DataFrame(encoded_data, columns=[col_name + '_'  + label for label in mlb.classes_])
    print(encoded_df)
    return encoded_df

In [64]:
# Passing the volumn values to the function

df_encoded_bm = clean_and_encode(df_elevate_gemi, 'BUSINESS_MODEL')
df_encoded_fs = clean_and_encode(df_elevate_gemi, 'FUNDING_SOURCES')
df_encoded_rm = clean_and_encode(df_elevate_gemi, 'REVENUE_MODEL')
df_encoded_in = clean_and_encode(df_elevate_gemi, 'INDUSTRIES')
df_encoded_th = clean_and_encode(df_elevate_gemi, 'TECHNOLOGY')

["['B2B', 'B2B2C']" "['B2B']" "['B2B', 'B2C']"
 "['B2B', 'B2B2C', 'B2C', 'B2G', 'C2C']" "['B2B', 'B2B2C', 'B2C']"
 "['B2B', 'B2B2C', 'B2G']" "['B2B2C']" "['B2B', 'B2G']" "['B2C']"
 "['B2B2C', 'B2C']" "['B2B', 'B2C', 'Other']"
 "['B2B', 'B2B2C', 'B2C', 'B2G']" "['B2B', 'B2B2C', 'C2C']"
 "['B2B2C', 'Other']" "['B2B', 'B2G', 'Other']"
 "['B2B', 'B2B2C', 'B2C', 'B2G', 'Other']"
 "['B2B', 'B2B2C', 'B2C', 'B2G', 'C2C', 'Other']" "['Other']" "['B2G']"
 "['B2B', 'B2C', 'B2G']" '[]' "['B2B2C', 'B2G']"
 "['B2B', 'B2B2C', 'B2C', 'Other']" "['B2B', 'Other']"
 "['B2B', 'B2C', 'C2C']" "['B2C', 'C2C']" "['B2C', 'B2G']"
 "['B2B', 'B2B2C', 'B2C', 'C2C']" "['C2C']" "['B2B2C', 'B2C', 'B2G']"]
0
["['B2B', 'B2B2C']" "['B2B']" "['B2B', 'B2C']"
 "['B2B', 'B2B2C', 'B2C', 'B2G', 'C2C']" "['B2B', 'B2B2C', 'B2C']"
 "['B2B', 'B2B2C', 'B2G']" "['B2B2C']" "['B2B', 'B2G']" "['B2C']"
 "['B2B2C', 'B2C']" "['B2B', 'B2C', 'Other']"
 "['B2B', 'B2B2C', 'B2C', 'B2G']" "['B2B', 'B2B2C', 'C2C']"
 "['B2B2C', 'Other']" "['B2B'

In [65]:
df_encoded = pd.concat([df_encoded_bm, df_encoded_fs, df_encoded_rm, df_encoded_in, df_encoded_th], axis=1)

# drop the original columns
#df_elevate_gemi = df_elevate_gemi.drop(['BUSINESS_MODEL', 'FUNDING_SOURCES', 'REVENUE_MODEL', 'INDUSTRIES', 'TECHNOLOGY'], axis=1)

# merge the encoded dataframe with the original dataframe
df_elevate_gemi = pd.concat([df_elevate_gemi, df_encoded], axis=1)


In [66]:
df_elevate_gemi.head(2)

,Unnamed:_0.1,Unnamed:_0_x,id,Corporate_Name_x,Region,INDUSTRIES,TECHNOLOGY,BUSINESS_MODEL,REVENUE_MODEL,FUNDING_SOURCES,...,TECHNOLOGY_Hardware,TECHNOLOGY_IoT,TECHNOLOGY_Nanotech,TECHNOLOGY_Networks,TECHNOLOGY_Other,TECHNOLOGY_Quantum Computing,TECHNOLOGY_Robotics,TECHNOLOGY_Sensors,TECHNOLOGY_Software,TECHNOLOGY_Web or Mobile Application
0,0,0,1,[i2.d] technologies,"Halandri, Greece",['Enterprise Software'],['Cloud Computing'],"['B2B', 'B2B2C']","['Commission Fee', 'SaaS', 'Subscription']",['VC'],...,0,0,0,0,0,0,0,0,0,0
1,1,1,10,A.D.W.K. ADWORK,"ZOGRAFOU , Greece",['Advertising & Marketing (AdTech)'],"['AI', 'AR/VR', 'Data Analytics - Big Data', '...","['B2B', 'B2B2C']","['Price per Item', 'SaaS']",['Self'],...,0,0,0,0,0,0,0,0,1,1


# Handling Dictionary

In [67]:
# replace null or empty values with default value
df_elevate_gemi['FUNDING'].fillna("[{'title': 'None', 'amount': 'None', 'year': '0', 'type': 'None'}]", inplace=True)

In [68]:
def extract_data(row):
    if isinstance(row, str):
        row = eval(row)  # convert string to list of dictionaries
    elif np.isnan(row):
        return pd.Series({'title': np.nan, 'amount': np.nan, 'year': np.nan, 'type': np.nan})
    latest_year = max(row, key=lambda x: int(x['year']))['year']
    latest_dict = [d for d in row if d['year'] == latest_year][0]
    # split title and type strings into lists, ignoring commas inside brackets and single quotes
    # title_list = re.findall(r"[\w\s-]+[\w\s−]+[\w\s−]+", latest_dict['title'])
    title_list = re.findall(r"[^',]+(?:[^',]+)*", latest_dict['title'])
    type_list = re.findall(r"[^',]+(?:[^',]+)*", latest_dict['type'])
    return pd.Series({'title': title_list, 'amount': latest_dict['amount'], 'year': latest_year, 'type': type_list})

In [69]:
# apply the function to the FUNDING column
df_elevate_gemi[['title', 'amount', 'year', 'type']] = df_elevate_gemi['FUNDING'].apply(extract_data)

# print the updated dataframe
df_elevate_gemi.head(6)

,Unnamed:_0.1,Unnamed:_0_x,id,Corporate_Name_x,Region,INDUSTRIES,TECHNOLOGY,BUSINESS_MODEL,REVENUE_MODEL,FUNDING_SOURCES,...,TECHNOLOGY_Other,TECHNOLOGY_Quantum Computing,TECHNOLOGY_Robotics,TECHNOLOGY_Sensors,TECHNOLOGY_Software,TECHNOLOGY_Web or Mobile Application,title,amount,year,type
0,0,0,1,[i2.d] technologies,"Halandri, Greece",['Enterprise Software'],['Cloud Computing'],"['B2B', 'B2B2C']","['Commission Fee', 'SaaS', 'Subscription']",['VC'],...,0,0,0,0,0,0,[None],None,0,[None]
1,1,1,10,A.D.W.K. ADWORK,"ZOGRAFOU , Greece",['Advertising & Marketing (AdTech)'],"['AI', 'AR/VR', 'Data Analytics - Big Data', '...","['B2B', 'B2B2C']","['Price per Item', 'SaaS']",['Self'],...,0,0,0,0,1,1,[None],None,0,[None]
2,2,2,100,ENEXAN P.C.,"Tripolis, Greece","['Environment & Energy (GreenTech, CleanTech)']","['AI', 'Cleantech', 'Software', 'Web or Mobile...","['B2B', 'B2B2C']","['Commission Fee', 'Licencing', 'Subscription']",['Self'],...,0,0,0,0,1,1,[None],None,0,[None]
3,3,3,101,ENFORGE PC,"Athens, Greece",['Manufacturing'],"['AI', 'Data Analytics - Big Data', 'Other', '...",['B2B'],"['SaaS', 'Subscription']","['Grant', 'Self']",...,1,0,0,0,1,1,[None],None,0,[None]
4,4,4,102,Engineers For Business S.A.,"Thessaloniki, Greece",['AgriTech / FoodTech'],"['Blockchain', 'Cloud Computing', 'Drones', 'I...",['B2B'],"['Other', 'Price per Item', 'SaaS', 'Subscript...","['Grant', 'Other', 'Self']",...,0,0,1,1,1,1,[None],None,0,[None]
5,5,5,103,ENGINIUS IKE,"Kallithea, Greece",['Advertising & Marketing (AdTech)'],"['AR/VR', 'Blockchain', 'Cloud Computing', 'Ne...","['B2B', 'B2C']","['Other', 'Price per Item', 'SaaS', 'Subscript...",['Self'],...,0,0,0,1,1,1,[ELEVATE],EUR 45K,2020,"[Series A, Other]"


In [70]:
# dictionary like preposcessing for similar columns

df_elevate_gemi['PATENTS_COUNT'] = df_elevate_gemi['PATENTS'].apply(lambda x: 0 if pd.isna(x) else x.count("'title'"))

df_elevate_gemi['AWARDS_COUNT'] = df_elevate_gemi['AWARDS'].apply(lambda x: 0 if pd.isna(x) else x.count("'title'"))

df_elevate_gemi['INCUBATIONS/ACCELERATIONS_COUNT'] = df_elevate_gemi['INCUBATIONS/ACCELERATIONS'].apply(lambda x: 0 if pd.isna(x) else x.count("'title'"))

# Label encoding

In [71]:
# create an instance of the LabelEncoder class
encoder = LabelEncoder()

# encode the 'Funding_State' column
df_elevate_gemi['Funding_State'] = encoder.fit_transform(df_elevate_gemi['Funding_State'])

# print the resulting dataframe with the encoded 'Funding_State' column
print(df_elevate_gemi)

     Unnamed:_0.1  Unnamed:_0_x   id  \
0               0             0    1   
1               1             1   10   
2               2             2  100   
3               3             3  101   
4               4             4  102   
..            ...           ...  ...   
599           599           599   95   
600           600           600   96   
601           601           601   97   
602           602           602   98   
603           603           603   99   

                                      Corporate_Name_x  \
0                                  [i2.d] technologies   
1                                      A.D.W.K. ADWORK   
2                                          ENEXAN P.C.   
3                                           ENFORGE PC   
4                          Engineers For Business S.A.   
..                                                 ...   
599  BUTLAIR Electronic Communication and Advertisi...   
600                                    BUY THE MILE SA 

In [72]:
value = df_elevate_gemi.loc[df_elevate_gemi['id'] == 101, 'Funding_State'].values[0]
print(value)

0


# Handling Gemi Preprocessing

In [73]:
# convert the columns to string type

df_elevate_gemi['value_of_share'] = df_elevate_gemi['value_of_share'].astype(str)
df_elevate_gemi['no_of_shares'] = df_elevate_gemi['no_of_shares'].astype(str)

# replace the euro symbol with a space in the 'value_of_share' column

df_elevate_gemi['value_of_share'] = df_elevate_gemi['value_of_share'].str.replace('€', '')
df_elevate_gemi['value_of_share'] = df_elevate_gemi['value_of_share'].str.replace(',', '')

# replace 'Ordinary' with 1 in the 'no_of_shares' column

df_elevate_gemi['no_of_shares'] = df_elevate_gemi['no_of_shares'].str.replace('Ordinary', '1')
df_elevate_gemi['no_of_shares'] = df_elevate_gemi['no_of_shares'].str.replace(',', '')

df_elevate_gemi['Date_Started_new'] = pd.to_datetime(df_elevate_gemi['Date_Started'],  format='%d/%m/%Y')

# Replace 'nan' strings with np.nan

df_elevate_gemi.replace('nan', np.nan, inplace=True)

# Convert the columns to numeric types

df_elevate_gemi['value_of_share'] = pd.to_numeric(df_elevate_gemi['value_of_share'])
df_elevate_gemi['no_of_shares'] = pd.to_numeric(df_elevate_gemi['no_of_shares'])

# Perform computations on the new dataframe

df_elevate_gemi['capital'] = df_elevate_gemi['value_of_share'] * df_elevate_gemi['no_of_shares']

# create a dictionary to map values to codes

legal_type_dict = {'Energy Community': 0, 'LLC':1, 'PC': 2, 'SA': 3, 'SOLE':4}
kind_of_shares_dict = {'Capital':0, 'Anonymous':0, 'non-Capital':1, 'Guarantee':2}

# use the dictionary to assign codes to the values in the Legal_Type column

df_elevate_gemi['Legal_Type_Code'] = df_elevate_gemi['Legal_Type'].map(legal_type_dict)
df_elevate_gemi['kind_of_shares_Code'] = df_elevate_gemi['kind_of_shares'].map(kind_of_shares_dict)

# REGION

In [74]:
# Code to lower and strip regions from 'Region' Column
df_elevate_gemi["Region"] = (df_elevate_gemi["Region"].str.lower()).str.replace(r', greece', '')

 # Code to replace all spelling errors in column values
value_corrections = {'ag paraskeyh': 'aghia paraskevi',
                    'aghia paraskevi, attikis': 'aghia paraskevi',
                    'aliartos viotia' : 'aliartos',
                    'analispi hersonisos': 'analispi',
                    'axarnes': 'acharnes',
                    'likovrisi - marousi ': 'lykovrysi',
                    'lycovrysi': 'lykovrysi',
                    'n. psyhiko athina': 'athina',
                    'neo psychico': 'neo',
                    'pilea': 'pylaia',
                    'pilea - thessaloniki': 'pylaia',
                    'pireas': 'piraeus',
                    'thesssaloniki': 'thessaloniki',
                    'thissions, athens': 'athens'}

df_elevate_gemi['Region'] = df_elevate_gemi['Region'].replace(value_corrections)

In [75]:
# Function to call api to extract region name
def get_greece_region_name(input_str):
  '''Function returns region name'''
  geolocator = Nominatim(user_agent="my_geocoder")
  location = geolocator.geocode(input_str + ", Greece", language="en")

  if location is not None:
    if 'Regional Unit' in location.raw['display_name'].split(',')[-3]:
      return location.raw['display_name'].split(',')[-2]
    else:
      return location.raw['display_name'].split(',')[-3]
  else:
      return input_str

In [76]:
# Code to replace all Region values
df_elevate_gemi["New_Region"] = df_elevate_gemi["Region"].apply(get_greece_region_name)

In [23]:
#df_elevate_gemi.to_excel("Cleaned_data.xlsx", index=False)

# Dropping Unused Columns

In [83]:
for col in df_elevate_gemi.columns:
  print(col)

id
Funding_State
region_population
Date_Started
BUSINESS_MODEL_B2B
BUSINESS_MODEL_B2B2C
BUSINESS_MODEL_B2C
BUSINESS_MODEL_B2G
BUSINESS_MODEL_C2C
BUSINESS_MODEL_Other
FUNDING_SOURCES_Angel
FUNDING_SOURCES_Crowd
FUNDING_SOURCES_Grant
FUNDING_SOURCES_Loan
FUNDING_SOURCES_Other
FUNDING_SOURCES_Self
FUNDING_SOURCES_VC
REVENUE_MODEL_Advertising
REVENUE_MODEL_Commission Fee
REVENUE_MODEL_Licencing
REVENUE_MODEL_Marketplace/Brokerage
REVENUE_MODEL_Other
REVENUE_MODEL_Price per Item
REVENUE_MODEL_Price per Use
REVENUE_MODEL_SaaS
REVENUE_MODEL_Subscription
INDUSTRIES_Advertising & Marketing (AdTech)
INDUSTRIES_AgriTech / FoodTech
INDUSTRIES_Art, Cultural & Creative Industries
INDUSTRIES_Data Analytics - Big Data
INDUSTRIES_Defence Systems
INDUSTRIES_EduTech - Education
INDUSTRIES_Enterprise Software
INDUSTRIES_Entertainment/Media (Games, Sports, Social)
INDUSTRIES_Environment & Energy (GreenTech, CleanTech)
INDUSTRIES_FinTech – Financial Services (WealthTech)
INDUSTRIES_Hardware
INDUSTRIES_Human

In [82]:
def drop_columns(df, columns_to_drop):
    return df.drop(columns_to_drop, axis=1)

In [81]:
"""columns_to_drop = ['Unnamed:_0.1','Unnamed:_0_x','ONE_LINER','About','Solution_&_Product','Innovation_&_Scalability',
'Traction_&_achievements','Team_&_Why_Us','Funding_Needs','Unnamed:_0_y','Corporate_Name_y','Commercial_Titles','Address',
'Website', 'Corporate_Name_x', 'Region', 'INDUSTRIES', 'TECHNOLOGY', 'BUSINESS_MODEL', 'REVENUE_MODEL', 'FUNDING_SOURCES', 'ONE_LINER', "About", 
  'Solution_&_Product', 'Innovation_&_Scalability', 'Traction_&_achievements', 'Team_&_Why_Us', 'Funding_Needs', 'FUNDING', 'PATENTS', 'AWARDS', 
  'INCUBATIONS/ACCELERATIONS', 'Legal_Type', 'kind_of_shares', 'no_of_shares', 'value_of_share', 'Date_Started','New_Region', 'Date_Started']
df_elevate_gemi = drop_columns(df_elevate_gemi, columns_to_drop)"""


KeyError: ignored

In [89]:
for col in df_elevate_gemi.columns:
  print(col)

id
Funding_State
region_population
Date_Started
BUSINESS_MODEL_B2B
BUSINESS_MODEL_B2B2C
BUSINESS_MODEL_B2C
BUSINESS_MODEL_B2G
BUSINESS_MODEL_C2C
BUSINESS_MODEL_Other
FUNDING_SOURCES_Angel
FUNDING_SOURCES_Crowd
FUNDING_SOURCES_Grant
FUNDING_SOURCES_Loan
FUNDING_SOURCES_Other
FUNDING_SOURCES_Self
FUNDING_SOURCES_VC
REVENUE_MODEL_Advertising
REVENUE_MODEL_Commission Fee
REVENUE_MODEL_Licencing
REVENUE_MODEL_Marketplace/Brokerage
REVENUE_MODEL_Other
REVENUE_MODEL_Price per Item
REVENUE_MODEL_Price per Use
REVENUE_MODEL_SaaS
REVENUE_MODEL_Subscription
INDUSTRIES_Advertising & Marketing (AdTech)
INDUSTRIES_AgriTech / FoodTech
INDUSTRIES_Art, Cultural & Creative Industries
INDUSTRIES_Data Analytics - Big Data
INDUSTRIES_Defence Systems
INDUSTRIES_EduTech - Education
INDUSTRIES_Enterprise Software
INDUSTRIES_Entertainment/Media (Games, Sports, Social)
INDUSTRIES_Environment & Energy (GreenTech, CleanTech)
INDUSTRIES_FinTech – Financial Services (WealthTech)
INDUSTRIES_Hardware
INDUSTRIES_Human

In [93]:
df_elevate_gemi['Date_Started_new'].dtype

dtype('<M8[ns]')

In [97]:
for col in df_elevate_gemi.columns:
  print(col)

id
Funding_State
region_population
Date_Started
BUSINESS_MODEL_B2B
BUSINESS_MODEL_B2B2C
BUSINESS_MODEL_B2C
BUSINESS_MODEL_B2G
BUSINESS_MODEL_C2C
BUSINESS_MODEL_Other
FUNDING_SOURCES_Angel
FUNDING_SOURCES_Crowd
FUNDING_SOURCES_Grant
FUNDING_SOURCES_Loan
FUNDING_SOURCES_Other
FUNDING_SOURCES_Self
FUNDING_SOURCES_VC
REVENUE_MODEL_Advertising
REVENUE_MODEL_Commission Fee
REVENUE_MODEL_Licencing
REVENUE_MODEL_Marketplace/Brokerage
REVENUE_MODEL_Other
REVENUE_MODEL_Price per Item
REVENUE_MODEL_Price per Use
REVENUE_MODEL_SaaS
REVENUE_MODEL_Subscription
INDUSTRIES_Advertising & Marketing (AdTech)
INDUSTRIES_AgriTech / FoodTech
INDUSTRIES_Art, Cultural & Creative Industries
INDUSTRIES_Data Analytics - Big Data
INDUSTRIES_Defence Systems
INDUSTRIES_EduTech - Education
INDUSTRIES_Enterprise Software
INDUSTRIES_Entertainment/Media (Games, Sports, Social)
INDUSTRIES_Environment & Energy (GreenTech, CleanTech)
INDUSTRIES_FinTech – Financial Services (WealthTech)
INDUSTRIES_Hardware
INDUSTRIES_Human

In [95]:
# Convert date column to ordinal integers
# replace NaT values with minimum datetime value
df_elevate_gemi['Date_Started_new'] = df_elevate_gemi['Date_Started_new'].fillna(df_elevate_gemi['Date_Started_new'].min())
df_elevate_gemi['Date_Started_new'] = df_elevate_gemi['Date_Started_new'].apply(lambda x: x.toordinal())

In [98]:
from sklearn.preprocessing import StandardScaler
#We transform (centralize) the entire X (independent variable data) to normalize it using standardscalar through transformation. We will create the PCA dimensions
# on this distribution. 
sc = StandardScaler()
df_elevate_gemi =  sc.fit_transform(df_elevate_gemi)   

ValueError: ignored